In [35]:
from __future__ import print_function
import numpy as np
import os
import h5py
import random
import itertools
import pandas as pd
import numpy as np
from collections import defaultdict
import pyfftw
import Pk_library as PKL

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cuda')

## Part 1: Read Catalogue Files

In [73]:
def read_halo_file(catalogue_path = "halo_catalogues/",
                   redshift = 0,
                  ):
    path = "{}Halo_catalogue_z={}.hdf5".format(catalogue_path, 
                                                redshift)
    
    f = h5py.File(path)
    
    mass = f["mass"]
    position = f["pos"]
    radius = f["radius"]
    
    return mass, position, radius

In [74]:
def build_redshift_vocab(catalogue_path = None,
                         redshift_list = range(6)):
    
    halo_data = {}
    
    for r in redshift_list:
        halo_data[r] = read_halo_file(catalogue_path= "halo_catalogues/",
                                       redshift = r)
        
    return halo_data

In [75]:
mass_z0, position_z0, radius_z0 = read_halo_file("halo_catalogues/", 0)

In [76]:
halo_data = build_redshift_vocab(catalogue_path = "halo_catalogues/", redshift_list = range(6))

## Part 2: Compute $M_{HI}$ for Given $M$

$$ M_{HI}(M, z) = M_0 (\frac{M}{M_{min}})^{\alpha}exp(-(\frac{M_{min}}{M})^{0.35}) $$

#### FoF Halos

"Friends-of-Friends"

In [51]:
num_redshifts = 5
param_data_fof = defaultdict(dict)

# lookup tables indexed by redshift
alpha_param_fof = [0.24,0.53,0.6,0.76,0.79,0.74]
m_0_fof = [4.3*1e10, 1.5*1e10, 1.3*1e10, 2.9*1e9, 1.4*1e9, 1.9*1e9]
m_min_fof = [2*1e12, 6*1e11, 3.6*1e11, 6.7*1e10, 2.1*1e10, 2*1e10]
m_hard_fof = [1.5*1e10, 6.9*1e9, 3.1*1e9, 9.9*1e8, 3.9*1e8, 2.7*1e8]

for r in range(num_redshifts + 1):
    param_data_fof[r]["alpha"] = alpha_param_fof[r]
    param_data_fof[r]["m_0"] = m_0_fof[r]
    param_data_fof[r]["m_min"] = m_min_fof[r]
    param_data_fof[r]["m_hard"] = m_hard_fof[r]

In [52]:
param_data_fof

defaultdict(dict,
            {0: {'alpha': 0.24,
              'm_0': 43000000000.0,
              'm_min': 2000000000000.0,
              'm_hard': 15000000000.0},
             1: {'alpha': 0.53,
              'm_0': 15000000000.0,
              'm_min': 600000000000.0,
              'm_hard': 6900000000.0},
             2: {'alpha': 0.6,
              'm_0': 13000000000.0,
              'm_min': 360000000000.0,
              'm_hard': 3100000000.0},
             3: {'alpha': 0.76,
              'm_0': 2900000000.0,
              'm_min': 67000000000.0,
              'm_hard': 990000000.0},
             4: {'alpha': 0.79,
              'm_0': 1400000000.0,
              'm_min': 21000000000.0,
              'm_hard': 390000000.0},
             5: {'alpha': 0.74,
              'm_0': 1900000000.0,
              'm_min': 20000000000.0,
              'm_hard': 270000000.0}})

#### FoF-SO Halos

"Friends-of-Friends Spherical Overdensity"

In [53]:
num_redshifts = 5
param_data_fofso = defaultdict(dict)

# lookup tables indexed by redshift
alpha_param_fofso = [0.16, 0.43, 0.51, 0.69, 0.61, 0.59]
m_0_fofso = [4.1*1e10, 1.8*1e10, 1.5*1e10, 3.7*1e9, 4.5*1e9, 4.1*1e9]
m_min_fofso = [2.4*1e12, 8.6*1e11, 4.6*1e11, 9.6*1e10, 7.6*1e10, 5.4*1e10]
m_hard_fofso = [1.3*1e10, 6.1*1e9, 2.5*1e9, 7.6*1e8, 2.3*1e8, 1.7*1e8]

for r in range(num_redshifts + 1):
    param_data_fofso[r]["alpha"] = alpha_param_fofso[r]
    param_data_fofso[r]["m_0"] = m_0_fofso[r]
    param_data_fofso[r]["m_min"] = m_min_fofso[r]
    param_data_fofso[r]["m_hard"] = m_hard_fofso[r]

In [54]:
param_data_fofso

defaultdict(dict,
            {0: {'alpha': 0.16,
              'm_0': 41000000000.0,
              'm_min': 2400000000000.0,
              'm_hard': 13000000000.0},
             1: {'alpha': 0.43,
              'm_0': 18000000000.0,
              'm_min': 860000000000.0,
              'm_hard': 6100000000.0},
             2: {'alpha': 0.51,
              'm_0': 15000000000.0,
              'm_min': 459999999999.99994,
              'm_hard': 2500000000.0},
             3: {'alpha': 0.69,
              'm_0': 3700000000.0,
              'm_min': 96000000000.0,
              'm_hard': 760000000.0},
             4: {'alpha': 0.61,
              'm_0': 4500000000.0,
              'm_min': 76000000000.0,
              'm_hard': 229999999.99999997},
             5: {'alpha': 0.59,
              'm_0': 4099999999.9999995,
              'm_min': 54000000000.0,
              'm_hard': 170000000.0}})

#### $M_{HI}$ Function: $$ M_{HI}(M, z) = M_0 (\frac{M}{M_{min}})^{\alpha}exp(-(\frac{M_{min}}{M})^{0.35}) $$

In [85]:
def MHI(m = None, redshift = 0, param_dict = param_data_fof):
    
    """Args:
    
    - m: (int) representing M, total mass in a given position,
    - redshift: (int) representing redshift
    - param_dict: (dict) param_data_fof or param_data_fofso
    
       Returns:
    
    - m_hi: (float) M_HI, total hydrogen(I) mass in a given 
            position
       """
    
    assert m != None, "Make sure to input M (mass)"
    assert redshift in range(6), "Redshift should be an integer in interval [0,5]"
    
    m_0, m_min = param_dict[redshift]["m_0"], param_dict[redshift]["m_min"]
    alpha = param_dict[redshift]["alpha"]
    
    # hard cutoff
    m_hard = param_dict[redshift]["m_hard"]
    
    x = m/m_min
    
    m_hi = m_0*(x**alpha)*(np.exp(-1*(1/x)**(0.35)))
    
    return m_hi

### Compute $M_{HI}$ for Each Redshift

In [77]:
def compute_mhi(redshift=None, dict_ = param_data_fof):
    
    assert redshift in range(6), "Redshift should be an integer in interval [0,5]"
    
    mass_data, position_data, radius_data = halo_data[redshift][0], halo_data[redshift][1], halo_data[redshift][2]
    m_hi = [MHI(m, redshift = redshift, param_dict = dict_) for m in np.array(mass_data)]
    return m_hi

In [93]:
MHI_data = {}

for r in range(num_redshifts+1):
    MHI_data[r] = compute_mhi(redshift = r, dict_ = param_data_fof)

In [94]:
[len(MHI_data[x]) for x in range(6)]

[4231400, 5081532, 5521701, 5587994, 5288375, 4693294]

In [95]:
for key in range(num_redshifts+1):
    pd.DataFrame(MHI_data[key]).to_csv("MHI_z={}.csv".format(key), index=False)